# <span style='color:blue'> *Blue Italic steps are optional (run if OSA connected)* </span>

In [1]:
from NKTP_DLL import *
import time
import numpy as np
import pandas as pd
from sys import getsizeof
import socket
import csv
import colorama
from colorama import Fore

Loading x64 DLL from: C:\Users\Public\Documents\NKT Photonics\SDK\NKTPDLL\x64\NKTPDLL.dll


## Connect NKT Select

In [2]:
print('Find modules on all existing and accessible ports - Might take a few seconds to complete.....')
if (getLegacyBusScanning()):
    print('Scanning following ports in legacy mode:', getAllPorts())
else:
    print('Scanning following ports in normal mode:', getAllPorts())

# Use the openPorts function with Auto & Live settings. This will scan and detect modules
# on all ports returned by the getAllPorts function.
# Please note that a port being in use by another application, will show up in this list but will
# not show any devices due to this port being inaccessible.
print(openPorts(getAllPorts(), 1, 1))

# All ports returned by the getOpenPorts function has modules (ports with no modules will automatically be closed)
print('Following ports has modules:', getOpenPorts())

# Traverse the getOpenPorts list and retrieve found modules via the deviceGetAllTypes function
portlist = getOpenPorts().split(',')
for portName in portlist:
    result, devList = deviceGetAllTypes(portName)
    print(result,type(devList))
    for devId in range(0, len(devList)):
        if (devList[devId] != 0):
#             print('Comport:',portName,'Device type:',"0x%0.2X" % devList[devId],'at address:',devId)
            if devList[devId]==0x60:
                extreme=devId
            elif devList[devId]==0x61:
                extreme_fp=devId
            elif devList[devId]==0x66:
                RF_power=devId
            elif devList[devId]==0x67:
                SuperK_select=devId
    print(f"device address:\nextreme@ {extreme},extreme_fp@ {extreme_fp},RF_power@ {RF_power},SuperK_select@ {SuperK_select} ")
closeResult = closePorts('')
print('Close result: ', PortResultTypes(closeResult))

Find modules on all existing and accessible ports - Might take a few seconds to complete.....
Scanning following ports in normal mode: COM4
0
Following ports has modules: COM4
0 <class 'bytes'>
device address:
extreme@ 15,extreme_fp@ 1,RF_power@ 27,SuperK_select@ 16 
Close result:  0:OPSuccess


## <span style='color:red'> <span style='background :yellow' >Andor and PM connect (NKT is TCP host) </span>

In [35]:
# Connection to Andor
HOST_andor='127.0.0.1'  #Andor Client C++ program "camera_operate"
PORT_andor=27015
s_andor=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s_andor.bind((HOST_andor,PORT_andor))
s_andor.listen(1)
andor_conn, andor_address = s_andor.accept()
print("Connection from: " + str(andor_address))

Connection from: ('127.0.0.1', 57072)


In [21]:
flag=s_andor.close()
if flag==None:
    print("Andor Closed Successfully")

Andor Closed Successfully


In [4]:
# Connection to power meter
HOST_PM='127.0.0.1'  #Power meter Labview program
PORT_PM=27020
s_PM=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s_PM.bind((HOST_PM,PORT_PM))
s_PM.listen(1)
PM_conn, PM_address = s_PM.accept()
print("Connection from: " + str(PM_address))

Connection from: ('127.0.0.1', 53249)


In [3]:
flag=s_PM.close()
if flag==None:
    print("Socket Closed Successfully")

Socket Closed Successfully


## NKT Function defs

In [3]:
def Extreme_turnON(comport, extreme, emissionpercent):
    status=registerRead(comport,extreme,0x66,-1)
    time.sleep(0.5)
    if int.from_bytes(status[1], "little") & 0x01 == 0x00:
        result = registerWriteU16(comport,extreme,0x31,0x01,-1)  #constant power mode
        print('Setting constant power mode - Extreme:', RegisterResultTypes(result))
        time.sleep(0.2)
        
        emission=emissionpercent*10
        result = registerWriteU16(comport, extreme, 0x37, emission, -1)
        print('Setting emission level - Extreme:', RegisterResultTypes(result))
        time.sleep(0.2)
        
        result = registerWriteU8(comport, extreme, 0x30, 0x03, -1)
        print('Setting emission ON - Extreme:', RegisterResultTypes(result))
        time.sleep(0.5)
        
        
def crystal_select(comport,RF_power,SuperK_select, crystal_num):
    #crystal num: 0- vis/nir, 1- nir/ir
    result = registerRead(comport,RF_power,0x75,-1)
    time.sleep(0.2)
    print(result)
    if int.from_bytes(result[1], "little") != (crystal_num+1):
        result = registerWriteU8(comport, RF_power, 0x30, 0x00, -1)
        print(result)
        time.sleep(0.2)
        result = registerWriteU8(comport, SuperK_select, 0x34, 0x00, -1)
        print(result)
        time.sleep(0.2)
        result = registerRead(comport,RF_power,0x75,-1)
        time.sleep(0.2)
        if int.from_bytes(result[1], "little") != (crystal_num+1):
            result = registerWriteU8(comport, SuperK_select, 0x34, 0x01, -1)
            print(result)
            time.sleep(0.2)
        result = registerWriteU8(comport, SuperK_select, 0x35, crystal_num, -1)
        print(result)
        time.sleep(0.2)
        result = registerRead(comport,RF_power,0x75,-1)
        time.sleep(0.2)
        print(result)
        
def RF_turnON(comport,RF_power):
    for i in range(8):
        result=registerWriteU16(comport, RF_power, 0xB0+i,0,-1) #set all channel amplitude to 0
        time.sleep(0.2)
        print(result)
    status=registerRead(comport,RF_power,0x66,-1)
    if int.from_bytes(status[1], "little") & 0x01 == 0x00:
        result = registerWriteU8(comport, RF_power, 0x30, 0x01, -1)
        print('Turn RF Power ON:', RegisterResultTypes(result))
        time.sleep(0.2)
        
def RF_turnOFF(comport,RF_power):
    result = registerWriteU8(comport, RF_power, 0x30, 0x00, -1)
    print('Turn off RF Power:', RegisterResultTypes(result))
    time.sleep(0.2)
    
def Extreme_turnOFF(comport, extreme):
    result = registerWriteU8(comport, extreme, 0x30, 0x00, -1)
    print('Setting emission OFF - Extreme:', RegisterResultTypes(result))
    time.sleep(0.2)

In [4]:
def set_nkt(comport, RF_power,wavelength):
    result = registerWriteU32(comport, RF_power, 0x90, wavelength, -1)
    time.sleep(0.2)
    return result
# result=registerWriteU16('COM5', RF_power, 0xB0,1000,-1)
# wl=int(670.5*1000)
# result = set_nkt('COM5',RF_power,wl)
# print(result)

## <span style='color:blue'> *OSA Function defs* </span>

In [5]:
def sock_send(sock,msg,receive=False,cond=None):
    sock.send(msg.encode()+b"\r\n")
    if receive:
        if cond != None:
            data = b'0'
            while data.decode("utf-8")[0] != cond:
                sock.send(b":stat:oper:even?\r\n")
                data = sock.recv(1024)

                time.sleep(0.1)
#             print(data)
            return data
        else:
            data = sock.recv(1024)
#             print(data)
            return data
    return None

def sock_send_arr(sock,msg_list,receive=False):
    ret = []
    for msg in msg_list:
        ret += [sock_send(sock,msg,receive)]
    return ret

def querry_data(sock,msg):
    sock.send(msg.encode()+b"\r\n")  
    strData2=b""
    i=0
    while True:
#         print(f"iter{i}")
        i=i+1
        strData2=sock.recv(1024)
#         print(strData2)
        if strData2.decode("utf-8")[0]=="+":
            break  
    return strData2.decode("utf-8")

## <span style='color:blue'> *OSA User Inputs- please match with Mono/Trestles OSA settings* </span>

In [6]:
###for nir- remember to switch crystal !!
# start_WL=740.0  #OSA window start
# stop_WL=812.0  #OSA window end-endpoint not executed
# RLevel_nW=2.0
# Resln=2.0
# # Spn=50
# Sampling_interval= 0.25
# Average_times=1

#**********remember to move ALL FILES (IMAGE, OSA,filelog) before switching crystal

###for VIS- remember to switch crystal !!
start_WL=565.0  #OSA window start
stop_WL=675.0  #OSA window end-endpoint not executed
RLevel_nW= 900.0
Resln=.5
# Spn=50
Sampling_interval= 0.1
Average_times=1

sensitivity="high1"  #Options- norm, mid, high1,2,3. generally use high1
smoothing="OFF"

## NKT User Inputs

In [34]:
#for nir- remember to switch crystal !!! 
# iters=8
# start_wl=752.0
# stop_wl=803.0 #end point not executed
# step_wl=1.0


"""##For vis- remember to switch crystal !!! 
iters=5
start_wl=550.0
stop_wl=651.0 #end point not executed
step_wl=1.0"""

## ZOE's PRACTICE ## For vis- remember to switch crystal !!! 
iters=1
start_wl=570.0
stop_wl=671.0 #end point not executed
step_wl=2.0


wl_list_unique=np.arange(start_wl,stop_wl,step_wl)
wl_list=[]
if iters>1:
    for i in range(iters):
        wl_list=np.concatenate((wl_list, wl_list_unique),axis=-1)
else:
    wl_list=wl_list_unique
    
print(len(wl_list))

filename='NKTselect_peaks_summary.csv'

comport= 'COM4'
## for test images

# dir_NKT='C:/Users/walrus/Desktop/walrus_git/walrus_python_codes/'

# NKT_matchpoints=pd.read_csv(dir_NKT+'matching WL points list.csv',skiprows=0,index_col=None)

# wl_list=NKT_matchpoints['NKTSetpoint'].to_numpy()


510


## Execution starts here

#### Extreme Turn ON

In [31]:
##always double check the PORT! It keeps changing
comport= 'COM4'

Extreme_turnON(comport,extreme,100)

Setting constant power mode - Extreme: 0:RegResultSuccess
Setting emission level - Extreme: 0:RegResultSuccess
Setting emission ON - Extreme: 0:RegResultSuccess


#### Crystal select

In [32]:
crystal_num=0 #0- vis/nir, 1- nir/ir
crystal_select(comport,RF_power,SuperK_select,crystal_num) 

(0, b'\x01')


#### RF power Turn ON

In [33]:
RF_turnON(comport,RF_power)

0
0
0
0
0
0
0
0
Turn RF Power ON: 0:RegResultSuccess


## <span style='color:blue'><span style='background :yellow' >*Main Loop (if OSA)* </span>

In [36]:
fields=['index','setpoint','peakX','peakY','centerWL','widthFWHM']

#Connect to OSA
HOST = '192.168.0.1'    # The remote server (OSA)
PORT = 10001              # The same port as used by the server (OSA)
s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))    
print("OSA connected")
data = sock_send_arr(s,["open \"anonymous\"", " "],receive=True)

OSA_data={}
for i,wl in enumerate(wl_list):
    if i==0:
        result=registerWriteU16(comport, RF_power, 0xB0,1000,-1)
        if result!=0:
            print("Failed to set channel amplitude!!!")
            break
    wl_int=int(wl*1000)
    result = set_nkt(comport,RF_power,wl_int)
    if result==0:
        
        #  andor send goes here
        flag_andor=andor_conn.send(b"go")
        
        
        print(f"wavelengthl --> {wl} nm") 
        data = sock_send_arr(s,["*RST","CFORM1",f":sens:wav:star {start_WL}nm",f":sens:wav:stop {stop_WL}nm"])
        data=sock_send_arr(s,["CFORM1",f":sens:band:res {Resln}nm",f":sens:sens {sensitivity}"])
        data=sock_send_arr(s,[f":sens:sett:smo {smoothing}",\
                                ":sens:sweep:points:auto off",f":sens:sweep:step {Sampling_interval}nm",\
                              ":sens:corr:rvel:med air"])
        data=sock_send_arr(s,[f":sens:aver:coun {Average_times}",":sens:chop OFF"])
        data=sock_send_arr(s,[":DISPLAY:TRACE:Y1:SPACING LIN",f":DISPLAY:TRACE:Y1:RLEVEL {RLevel_nW}nW"])
        OSA_data['index']=i
        OSA_data['setpoint']=wl
        data = sock_send_arr(s,[":init:smode 1","*CLS",":init"])
        data = sock_send(s,":stat:oper:even?",receive=True,cond='1')
        data = sock_send_arr(s,["CFORM1",":mmem:cdr int"])
        
        
        #wait for andor
        msg_andor=andor_conn.recv(1024)
        if msg_andor.rstrip(b'\x00').decode("utf-8")=="abc": 
            #sending save signal to OSA
            print('andor ok')    
        else:
            print('PROBLEM IN ANDOR!!')
            break
                
        data = sock_send(s,f":mmem:stor:trac tra,csv,\"trace_{i}\",int",receive=True)
        
        ##calculate peak WL
        data=sock_send(s,":calc:mark:max")
        data_peakx=querry_data(s,":calc:mark:x? 0")        
        OSA_data['peakX']=float(data_peakx)*1e9
        print(f"peak={float(data_peakx)*1e9}")
        data_peaky=querry_data(s,":calc:mark:y? 0")
        OSA_data['peakY']=float(data_peaky)*1e9
        
        #calculate 3dB BW
        data=sock_send_arr(s,[":calc:cat swth"])
        data=sock_send_arr(s,[":calc:par:cat:swth:TH 3.0"])
        data=sock_send(s,":calc")
        str_thresh=querry_data(s,":calc:data?")
        data_thresh=str_thresh.split(',')
        OSA_data['centerWL']=float(data_thresh[0])*1e9
        OSA_data['widthFWHM']=float(data_thresh[1])*1e9
        
        if i==0:
            with open(filename, 'a',newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames = fields) 
                writer.writeheader() 
                
        with open(filename, 'a',newline='') as csvfile: 
    # creating a csv dict writer object 
            writer = csv.DictWriter(csvfile, fieldnames = fields) 
            writer.writerow(OSA_data) 
        time.sleep(1)
    else:
        print(f"Failed to change wavelength at {wl}")
        break
    
flag=s.close()
##################if andor is ON, remember to close socket from above
flag_andor=s_andor.close()
if flag==None:
    print("OSA Socket Closed Successfully")

OSA connected
wavelengthl --> 570.0 nm
andor ok
peak=569.5
wavelengthl --> 572.0 nm
andor ok
peak=571.5
wavelengthl --> 574.0 nm
andor ok
peak=573.5
wavelengthl --> 576.0 nm
andor ok
peak=575.5
wavelengthl --> 578.0 nm
andor ok
peak=577.5
wavelengthl --> 580.0 nm
andor ok
peak=579.5
wavelengthl --> 582.0 nm
andor ok
peak=581.5
wavelengthl --> 584.0 nm
andor ok
peak=583.5
wavelengthl --> 586.0 nm
andor ok
peak=585.5
wavelengthl --> 588.0 nm
andor ok
peak=587.5
wavelengthl --> 590.0 nm
andor ok
peak=589.5
wavelengthl --> 592.0 nm
andor ok
peak=591.5
wavelengthl --> 594.0 nm
andor ok
peak=593.5
wavelengthl --> 596.0 nm
andor ok
peak=595.5
wavelengthl --> 598.0 nm
andor ok
peak=597.5
wavelengthl --> 600.0 nm
andor ok
peak=599.5
wavelengthl --> 602.0 nm
andor ok
peak=601.5
wavelengthl --> 604.0 nm
andor ok
peak=603.6
wavelengthl --> 606.0 nm
andor ok
peak=605.5
wavelengthl --> 608.0 nm
andor ok
peak=607.6
wavelengthl --> 610.0 nm
andor ok
peak=609.5999999999999
wavelengthl --> 612.0 nm
ando

andor ok
peak=611.6
wavelengthl --> 614.0 nm
andor ok
peak=613.7
wavelengthl --> 616.0 nm
andor ok
peak=615.6999999999999
wavelengthl --> 618.0 nm
andor ok
peak=617.7
wavelengthl --> 620.0 nm
andor ok
peak=619.6999999999999
wavelengthl --> 622.0 nm
andor ok
peak=621.8000000000001
wavelengthl --> 624.0 nm
andor ok
peak=623.8
wavelengthl --> 626.0 nm
andor ok
peak=625.9
wavelengthl --> 628.0 nm
andor ok
peak=627.9
wavelengthl --> 630.0 nm
andor ok
peak=629.9
wavelengthl --> 632.0 nm
andor ok
peak=632.0
wavelengthl --> 634.0 nm
andor ok
peak=634.0
wavelengthl --> 636.0 nm
andor ok
peak=636.1
wavelengthl --> 638.0 nm
andor ok
peak=638.1
wavelengthl --> 640.0 nm
andor ok
peak=640.2
wavelengthl --> 642.0 nm
andor ok
peak=642.1999999999999
wavelengthl --> 644.0 nm
andor ok
peak=644.3000000000001
wavelengthl --> 646.0 nm
andor ok
peak=646.4
wavelengthl --> 648.0 nm
andor ok
peak=648.4
wavelengthl --> 650.0 nm
andor ok
peak=650.5
wavelengthl --> 652.0 nm
andor ok
peak=652.5
wavelengthl --> 654.

peak=654.6
wavelengthl --> 656.0 nm
andor ok
peak=656.6
wavelengthl --> 658.0 nm
andor ok
peak=658.7
wavelengthl --> 660.0 nm
andor ok
peak=660.8
wavelengthl --> 662.0 nm
andor ok
peak=662.9000000000001
wavelengthl --> 664.0 nm
andor ok
peak=664.9
wavelengthl --> 666.0 nm
andor ok
peak=667.0
wavelengthl --> 668.0 nm
andor ok
peak=669.0
wavelengthl --> 670.0 nm
andor ok
peak=671.1
wavelengthl --> 570.0 nm
andor ok
peak=569.6
wavelengthl --> 572.0 nm
andor ok
peak=571.5
wavelengthl --> 574.0 nm
andor ok
peak=573.6
wavelengthl --> 576.0 nm
andor ok
peak=575.6
wavelengthl --> 578.0 nm
andor ok
peak=577.6
wavelengthl --> 580.0 nm
andor ok
peak=579.6
wavelengthl --> 582.0 nm
andor ok
peak=581.6
wavelengthl --> 584.0 nm
andor ok
peak=583.6
wavelengthl --> 586.0 nm
andor ok
peak=585.5999999999999
wavelengthl --> 588.0 nm
andor ok
peak=587.6
wavelengthl --> 590.0 nm
andor ok
peak=589.6
wavelengthl --> 592.0 nm
andor ok
peak=591.6
wavelengthl --> 594.0 nm
andor ok
peak=593.6
wavelengthl --> 596.

In [17]:
if flag==None:
    print("OSA Socket Closed Successfully")

OSA Socket Closed Successfully


## <span style='color:red'>Main Loop (if no OSA) </span>

In [ ]:
for i,wl in enumerate(wl_list):
    if i==0:
        result=registerWriteU16('COM5', RF_power, 0xB0,1000,-1)
#       print(RegisterResultTypes(result))
        if result!=0:
            print("Failed to set channel amplitude!!!")
            break
    wl_int=int(wl*1000)
    result = set_nkt('COM5',RF_power,wl_int)
    if result==0:
        print(f"wavelengthl --> {wl} nm") 
#     result = registerRead('COM5',RF_power,0x66,-1)
#     print(result)
        
    time.sleep(1)


## <span style='color:red'>Main Loop (Andor + Powermeter) </span> 

##### This is <span style='color:red'> Option 1: </span> cont. from script "OSA_Monochromator_loop_Nafiz_v1.ipynb". Also see Option 2 there 

In [13]:
#User input 

powers= [300]
wavelegth= 670

iter_size=len(str(len(powers)).encode())
image_size=len(str(len(wl_list)).encode())
max_len=4+1+iter_size+1+image_size

print("for ANDOR sript info:")
print(f"no. of iterations: {len(powers)}")
print(f"no. of iamges: {len(wl_list)}")


print(f"max len: {max_len}")
def msgpadding(max_len,msg_list):
    
    msg_str=''
    msg_len=0
    for s in msg_list:
        msg_str+=s
    
    msg_len=len(msg_str.encode())
    if msg_len<max_len:
        msg_pad=msg_str.zfill(max_len)
    else:
        msg_pad=msg_str
    return msg_pad


#NKT image indexing for andor files
fields = ['iter','img_filename','NKT_setopint']
filename = "NKTselect_Andor_image_indexing.csv"
with open(filename, 'w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = fields) 
    writer.writeheader() 

for ANDOR sript info:
no. of iterations: 1
no. of iamges: 51
max len: 9


In [21]:
# result=0

for i,power in enumerate(powers):
    print("waiting to connect PM...")

    # Connection to power meter
    HOST_PM='127.0.0.1'  #Power meter Labview program
    PORT_PM=27020
    s_PM=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s_PM.bind((HOST_PM,PORT_PM))
    s_PM.listen(1)
    PM_conn, PM_address = s_PM.accept()
    if str(andor_address)!="":
        print("Connection from: " + str(PM_address))
        nm=input("Press start loop in Labview, then y in python")
        for j,wl in enumerate(wl_list):
            if j==0 and i==0:
                result=registerWriteU16('COM5', RF_power, 0xB0,1000,-1)
        #       print(RegisterResultTypes(result))
                if result!=0:
                    print("Failed to set channel amplitude!!!")
                    break
                    break
            wl_int=int(wl*1000)
            result = set_nkt('COM5',RF_power,wl_int)
            if result!=0:
                print("Failed to set wavelength!!!")
                break
                break           
            else:   

        #  andor send goes here
                flag_andor=andor_conn.send(b"go")
    # sending msg to PM       
                msg=msgpadding(max_len,str(round(wl,1))).encode()
                print(str(round(wl,1)).encode())
                print(msg)
                signal_PM=PM_conn.send(msg)

    # primary msg padding for PM, has to be 10bytes total
                if len(str(i).encode())<iter_size:
                    itr=msgpadding(iter_size,str(i))
                else:
                    itr=str(i)
                if len(str(j).encode())<image_size:
                    imag=msgpadding(image_size,str(j))
                else:
                    imag=str(j)

                print(f'waiting for andor signal')
    #andor recv goes here
                msg_andor=andor_conn.recv(1024)
                if msg_andor.rstrip(b'\x00').decode("utf-8")=="abc":            
        #sending stop msg to PM
                    if j==len(wl_list)-1:
                        msg=msgpadding(max_len,["endd,",itr,",",imag]).encode()
                        signal_PM=PM_conn.send(msg)  
                    else:
                        msg=msgpadding(max_len,["stop,",itr,",",imag]).encode()
                        signal_PM=PM_conn.send(msg)
                    msg_PM=PM_conn.recv(1024)
        #writing csv log file
                    if msg_PM==b'done':
                        with open(filename, 'a',newline='') as csvfile:
                            writer = csv.writer(csvfile) 
                            writer.writerow([power,f"iter_{i}_image_{j}",wl]) 
                        print(f"imgpow_taken- power --> {power},  wavelengthl --> {round(wl,1)} nm")
        flag=s_PM.close()
        if flag==None:
            print("--------------PM Socket Closed, ready for next iter---------------------")
print('finished!')

waiting to connect PM...
Connection from: ('127.0.0.1', 54421)
Press start loop in Labview, then y in pythony
b'655.8'
b'0000655.8'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 655.8 nm
b'656.8'
b'0000656.8'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 656.8 nm
b'657.8'
b'0000657.8'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 657.8 nm
b'658.2'
b'0000658.2'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 658.2 nm
b'658.7'
b'0000658.7'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 658.7 nm
b'659.6'
b'0000659.6'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 659.6 nm
b'662.2'
b'0000662.2'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 662.2 nm
b'663.6'
b'0000663.6'
waiting for andor signal
imgpow_taken- power --> 300,  wavelengthl --> 663.6 nm
b'664.0'
b'0000664.0'
waiting for andor signal
imgpow_taken- power --> 300

## <span style='color:red'> Option 2: </span> Send signal to Andor (for test images)

This part manually sends "go" to Andor and PM. Make sure Option 2 in the other ipynb is already executed before this

In [14]:
#Option2 log file
fields = ['NKT_setopint','img_filename','power filename']
filename = "NKTselect_Option2_filelog.csv"
with open(filename, 'w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = fields) 
    writer.writeheader() 

def msgpadding(max_len,msg_list):   
    msg_str=''
    msg_len=0
    for s in msg_list:
        msg_str+=s
    
    msg_len=len(msg_str.encode())
    if msg_len<max_len:
        msg_pad=msg_str.zfill(max_len)
    else:
        msg_pad=msg_str
    return msg_pad

In [15]:
#User input in each step
wl=  670.0 # NKTsetpoint
imagg=0
end_msg="endd,"

#don't change below
#assuming PM receives 9 bytes. Otherwise change iter_size
max_len=9
imag_size=2
iterr=0
iter_size=1

In [16]:
#setting NKTSelect WL
result=registerWriteU16(comport, RF_power, 0xB0,1000,-1)
# print(RegisterResultTypes(result))
if result!=0:
    print("Failed to set channel amplitude!!!")
else:
    wl_int=int(wl*1000)
    result = set_nkt(comport,RF_power,wl_int)
    if result!=0:
        print("Failed to set wavelength!!!")
    else:
        print("WL set")

WL set


In [70]:
# Connection to power meter
HOST_PM='127.0.0.1'  #Power meter Labview program
PORT_PM=27020
s_PM=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s_PM.bind((HOST_PM,PORT_PM))
s_PM.listen(1)
PM_conn, PM_address = s_PM.accept()
print("Connection from: " + str(PM_address))

Connection from: ('127.0.0.1', 53844)


## Sunlight- use this for test images !!!

make sure to start a fresh andor tcp

In [160]:
#  andor send goes here
flag_andor=andor_conn.send(b"go")

# # sending msg to PM       
# msg=msgpadding(max_len,str(round(wl,1))).encode()
# signal_PM=PM_conn.send(msg)

# # primary msg padding for PM, has to be 10bytes total
# itr=msgpadding(iter_size,str(iterr))
# imag=msgpadding(imag_size,str(imagg))

msg_andor=andor_conn.recv(1024)

if msg_andor.rstrip(b'\x00').decode("utf-8")=="abc": 
    print('andor ok')
#     msg=msgpadding(max_len,[end_msg,itr,",",imag]).encode()
#     signal_PM=PM_conn.send(msg) 
#     msg_PM=PM_conn.recv(1024)
#     if msg_PM==b'done':
#     with open(filename, 'a',newline='') as csvfile:
#         writer = csv.writer(csvfile) 
#         writer.writerow([wl,f"iter_{itr}_image_{imag}",f"iter_{itr}_image_{imag}"]) 
#     print("imgpow_taken !")


andor ok


## <span style='color:red'> Option 3: </span> Simultaneous Andor, OSA, PM

If Kernel dies in the middle: 
a) Move all andor files to a 'take 1 folder'  b) copy all OSA traces and move them to take1 folder. c) move the files 'NKTselect_Option3_filelog.csv' and 'NKTselect_peaks_summary' to take1 folder. d) move the PMLog files to take1 folder. 

e) restart code, change the NKTSelect starting point (just below) from where you crashed. you can later save these into a take2 folder

In [10]:
##Run only once
#NKT array
start_wl=659.0
stop_wl=680.0 #included in list
# stop_wl=658.1
step_wl=0.1
wl_dict={}

wl_list=np.arange(start_wl,stop_wl,step_wl)
# idx_list=np.arange(0,len(wl_list))

for i,wl in enumerate(wl_list):
    wl_dict[i]= wl
    
print(len(wl_dict))

210


In [8]:
##Run only once
#testfile log
fieldstest = [\
              'NKT_peakX','NKT_setpoint','img_filename','power filename', ]
filenametest = "NKTselect_Option3_filelog.csv"
with open(filenametest, 'w',newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = fieldstest) 
    writer.writeheader() 

# NKTSelect loop log 
fieldsloop = ['index','setpoint','peakX', 'peakY', 'centerWL', 'widthFWHM']
filenameloop = "NKTselect_peaks_summary.csv"


def msgpadding(max_len,msg_list):   
    msg_str=''
    msg_len=0
    for s in msg_list:
        msg_str+=s
    
    msg_len=len(msg_str.encode())
    if msg_len<max_len:
        msg_pad=msg_str.zfill(max_len)
    else:
        msg_pad=msg_str
    return msg_pad



In [33]:
##Run only once
#fetching mono peaks info
dir_mono_peakx='C:/Users/walrus/Desktop/spectrometer/Monochromator to MMI_2023_01_29/iHR550/'
mono_peaks=pd.read_csv(dir_mono_peakx+'monochromator peaks_summary_combined.csv',skiprows=1,index_col='mono_index', names=['mono_index','mono_peakX','mono_peakY','mono_centerWL','mono_widthFWHM'])

mono_setpoints=pd.read_csv(dir_mono_peakx+'testin_combined.csv',skiprows=0,names=["setpoint"],usecols=[0])
# mono_outpoints=pd.read_csv(dir_mono_peakx+'testout_OSA.txt',skiprows=0,names=["setpoint"],usecols=[0])


mono_peaks.insert(0,'mono_setpoints',mono_setpoints)
# mono_peaks.insert(1,'mono_outpoints',mono_outpoints)
# print(mono_peaks.loc[2]['mono_peakX'].dtype)
# print(mono_peaks['mono_peakX'].values.round(1))


dir_MMIpower='C:/Users/walrus/Desktop/spectrometer/Monochromator to MMI_2023_01_29/iHR550/'
MMI_power=pd.read_csv(dir_MMIpower+'avg powers.csv',skiprows=0,names=["mono_outpoints", "iter", "img", "power"])


itera=MMI_power['iter'].max()+1
imag=MMI_power['img'].max()+1
Lastpowers=pd.Series(MMI_power[MMI_power['iter']==itera-1]['power'])
Lastpowers.index=MMI_power[MMI_power['iter']==itera-1]['img']

print(MMI_power)

Avgpower=np.zeros((imag))
for j in range(imag):
    Avgpower[j]=np.mean(MMI_power[MMI_power['img']==j]['power'])

mono_peaks['mono_Avgpowers']=pd.Series(Avgpower)
# print(Avgpower)

mono_peaks['mono_Lastpowers'] = Lastpowers.values
mono_peaks.tail()


     mono_outpoints  iter  img        power
0           654.965     0    0   769.674141
1           655.500     0    1   986.230240
2           656.000     0    2   774.801465
3           656.500     0    3   783.808154
4           656.999     0    4   753.657701
..              ...   ...  ...          ...
918         656.974     0  919   953.076468
919         656.473     0  920  1101.744099
920         655.975     0  921   967.952618
921         655.476     0  922   971.610607
922         654.976     0  923   822.415847

[923 rows x 4 columns]


ValueError: Length of values (923) does not match length of index (924)

In [14]:
#restart loop
new_index=0

<span style='color:red'> <b> Loop Start <b> </span> 

In [21]:
##RUn before each loop
#Connect to OSA
HOST = '192.168.0.1'    # The remote server (OSA)
PORT = 10001              # The same port as used by the server (OSA)
s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))    
print("OSA connected")
data = sock_send_arr(s,["open \"anonymous\"", " "],receive=True)

OSA_data={}

OSA connected


In [22]:
#loop
mono_matched_info={}
for i in range (new_index, len(wl_dict)):
# for i,wl in enumerate(wl_list):
    if i==new_index:
        result=registerWriteU16('COM5', RF_power, 0xB0,1000,-1)
        if result!=0:
            print("Failed to set channel amplitude!!!")
            break
    wl=wl_dict[i]
    wl_int=int(wl*1000)
    result = set_nkt('COM5',RF_power,wl_int)
    if result!=0:
        print("Failed to set wavelength!!!")
    else:
#         print(f"NKTSetpoint --> {wl} nm") 
        data = sock_send_arr(s,["*RST","CFORM1",f":sens:wav:star {start_WL}nm",f":sens:wav:stop {stop_WL}nm"])
        data=sock_send_arr(s,["CFORM1",f":sens:band:res {Resln}nm",f":sens:sens {sensitivity}"])
        data=sock_send_arr(s,[f":sens:sett:smo {smoothing}",\
                                ":sens:sweep:points:auto off",f":sens:sweep:step {Sampling_interval}nm",\
                              ":sens:corr:rvel:med air"])
        data=sock_send_arr(s,[f":sens:aver:coun {Average_times}",":sens:chop OFF"])
        data=sock_send_arr(s,[":DISPLAY:TRACE:Y1:SPACING LIN",f":DISPLAY:TRACE:Y1:RLEVEL {RLevel_nW}nW"])
        OSA_data['index']=i
        OSA_data['setpoint']=wl
        data = sock_send_arr(s,[":init:smode 1","*CLS",":init"])
        data = sock_send(s,":stat:oper:even?",receive=True,cond='1')
        data = sock_send_arr(s,["CFORM1",":mmem:cdr int"])
        data = sock_send(s,f":mmem:stor:trac tra,csv,\"trace_{i}\",int",receive=True)
        
        ##calculate peak WL
        data=sock_send(s,":calc:mark:max")
        data_peakx=querry_data(s,":calc:mark:x? 0")        
        OSA_data['peakX']=float(data_peakx)*1e9
        print(f"NKTSetpoint --> {wl} nm, NKTpeakX={float(data_peakx)*1e9}")
        data_peaky=querry_data(s,":calc:mark:y? 0")
        OSA_data['peakY']=float(data_peaky)*1e9
        
        #calculate 3dB BW
        data=sock_send_arr(s,[":calc:cat swth"])
        data=sock_send_arr(s,[":calc:par:cat:swth:TH 3.0"])
        data=sock_send(s,":calc")
        str_thresh=querry_data(s,":calc:data?")
        data_thresh=str_thresh.split(',')
        OSA_data['centerWL']=float(data_thresh[0])*1e9
        OSA_data['widthFWHM']=float(data_thresh[1])*1e9
        
        if i==0:
            with open(filenameloop, 'w',newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames = fieldsloop) 
                writer.writeheader() 
                
        with open(filenameloop, 'a',newline='') as csvfile: 
    # creating a csv dict writer object 
            writer = csv.DictWriter(csvfile, fieldnames = fieldsloop) 
            writer.writerow(OSA_data) 
        time.sleep(0.5)
        
        
        if round(OSA_data['peakX'],1) in mono_peaks['mono_peakX'].values.round(1):
            match_index=mono_peaks.index[mono_peaks['mono_peakX'].round(1)==round(OSA_data['peakX'],1)].tolist()
            #assuming only 1 match
            mono_matched_info['mono_index']=match_index
            mono_matched_info['mono_setpoint']=mono_peaks.loc[match_index]['mono_setpoints'].values
            mono_matched_info['mono_outpoint']=mono_peaks.loc[match_index]['mono_outpoints'].values
            mono_matched_info['mono_peakX']=mono_peaks.loc[match_index]['mono_peakX'].values
            mono_matched_info['mono_lastpower']=mono_peaks.loc[match_index]['mono_Lastpowers'].values
            print('found!======\
            mono_setpoint= %f,mono_peakX= %f, mono_lastpower= %f'\
                  %(mono_matched_info['mono_setpoint'],mono_matched_info['mono_peakX'],mono_matched_info['mono_lastpower']))
#             print(f"NKTSetpoint --> {wl} nm, NKTpeakX={float(data_peakx)*1e9}")
            print(Fore.RED+'NKTSetpoint --> %f nm, mono_lastpower= %f' %(wl, mono_matched_info['mono_lastpower']))
            break

NameError: name 'new_index' is not defined

<span style='color:brown'> <b> Single snap <b> </span> 

## <span style='color:red'> Xenon

In [6]:
# ###for trestles settings- low-res BF experiment
# start_WL=773
# stop_WL=816  #end point not executed

# Resln=0.1
# Sampling_interval= 0.01
# Average_times=1
# sensitivity="high1"  #Options- norm, mid, high1,2,3. generally use high1
# smoothing="OFF"

###for trestles settings- high-res ET2 experiment
start_WL=793
stop_WL=853  #end point not executed

Resln=0.5
Sampling_interval= 0.05
Average_times=1
sensitivity="high2"  #Options- norm, mid, high1,2,3. generally use high1
smoothing="OFF"

## <span style='color:red'> Xenon

In [7]:
#tresltes
# RLevel_nW=200 #DBR laser

# RLevel_nW=650 #DBR laser, ET2
RLevel_nW=30 #NKT

## <span style='color:red'> Xenon

In [9]:
#Run after a match found

#trestles
#########User input/ test filename 
wl=  100 # NKTsetpoint


imagg=0  #check from_andor_script for current image number


end_msg="endd,"

#don't change below
#assuming PM receives 9 bytes. Otherwise change iter_size
max_len=9
imag_size=2
iterr=0
iter_size=1

In [19]:
#set NKT wavelength
result=registerWriteU16('COM5', RF_power, 0xB0,1000,-1)
if result!=0:
    print("Failed to set channel amplitude!!!")
else:
    wl_int=int(wl*1000)
    result = set_nkt('COM5',RF_power,wl_int)
    if result!=0:
        print("Failed to set wavelength!!!")
    else:
        print("WL set")
########### adjust power wheel after this step        

WL set


## <span style='color:red'> Xenon 

In [194]:
imagg=83

In [195]:
# Connection to power meter
# HOST_PM='127.0.0.1'  #Power meter Labview program
# PORT_PM=27020
# s_PM=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
# s_PM.bind((HOST_PM,PORT_PM))
# s_PM.listen(1)
# PM_conn, PM_address = s_PM.accept()
# print("Connection from: " + str(PM_address))

#Connect to OSA
HOST = '192.168.0.1'    # The remote server (OSA)
PORT = 10001              # The same port as used by the server (OSA)
s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))    
print("OSA connected")
data = sock_send_arr(s,["open \"anonymous\"", " "],receive=True)


OSA connected


## <span style='color:red'> Xenon

In [196]:
#making sure labview is ready (most common mistake)
nm=input("Press start loop in Labview, then y in python")

#  sending go to andor
flag_andor=andor_conn.send(b"go")

# sending go to PM       
# msg=msgpadding(max_len,str(round(wl,1))).encode()
# signal_PM=PM_conn.send(msg)

#sending go to OSA
test_OSA_data={}

data = sock_send_arr(s,["*RST","CFORM1",f":sens:wav:star {start_WL}nm",f":sens:wav:stop {stop_WL}nm"])
data=sock_send_arr(s,["CFORM1",f":sens:band:res {Resln}nm",f":sens:sens {sensitivity}"])
data=sock_send_arr(s,[f":sens:sett:smo {smoothing}",\
                                ":sens:sweep:points:auto off",f":sens:sweep:step {Sampling_interval}nm",\
                              ":sens:corr:rvel:med air"])
data=sock_send_arr(s,[f":sens:aver:coun {Average_times}",":sens:chop OFF"])

#**********must check if RLevel looks good enough********
data=sock_send_arr(s,[":DISPLAY:TRACE:Y1:SPACING LIN",f":DISPLAY:TRACE:Y1:RLEVEL {RLevel_nW}nW"])
data = sock_send_arr(s,[":init:smode 1","*CLS",":init"])
data = sock_send(s,":stat:oper:even?",receive=True,cond='1')
data = sock_send_arr(s,["CFORM1",":mmem:cdr int"])
# data = sock_send(s,f":mmem:stor:trac tra,csv,\"test_{imagg}\",int",receive=True)


# primary msg padding for PM, has to be 10bytes total
itr=msgpadding(iter_size,str(iterr))
imag=msgpadding(imag_size,str(imagg))

msg_andor=andor_conn.recv(1024)
# bla=1
# if bla==1:
#     time.sleep(1)

if msg_andor.rstrip(b'\x00').decode("utf-8")=="abc": 
    #sending save signal to OSA
    print('andor ok')
    data = sock_send(s,f":mmem:stor:trac tra,csv,\"test_{imagg}\",int",receive=True)
    
    ##calculate peak WL
    data=sock_send(s,":calc:mark:max")
    data_peakx=querry_data(s,":calc:mark:x? 0")        
    test_OSA_data['peakX']=float(data_peakx)*1e9
    print(f"NKTSetpoint --> {wl} nm, peakX={float(data_peakx)*1e9}")
    data_peaky=querry_data(s,":calc:mark:y? 0")
    test_OSA_data['peakY']=float(data_peaky)*1e9

    #calculate 3dB BW
    data=sock_send_arr(s,[":calc:cat swth"])
    data=sock_send_arr(s,[":calc:par:cat:swth:TH 3.0"])
    data=sock_send(s,":calc")
    str_thresh=querry_data(s,":calc:data?")
    data_thresh=str_thresh.split(',')
    test_OSA_data['centerWL']=float(data_thresh[0])*1e9
    test_OSA_data['widthFWHM']=float(data_thresh[1])*1e9
    
    
    
    #sending end signal to PM
#     msg=msgpadding(max_len,[end_msg,itr,",",imag]).encode()
#     signal_PM=PM_conn.send(msg) 
#     msg_PM=PM_conn.recv(1024)
#     if msg_PM==b'done':
    with open(filenametest, 'a',newline='') as csvfile:
        writer = csv.writer(csvfile) 
        writer.writerow([\
                         test_OSA_data['peakX'],wl,\
                         f"iter_{itr}_image_{imag}",f"iter_{itr}_image_{imag}"]) 
        print("imgpow_taken")
#     else:
#         print("POWER METER PROBLEM !")
else:
    print("NO SIGNAL FROM ANDOR !")
    
nm=input("Press shutdown in Labview, then y in python")

Press start loop in Labview, then y in pythony
andor ok
NKTSetpoint --> 100 nm, peakX=579.6800000000001
imgpow_taken
Press shutdown in Labview, then y in pythony


In [220]:
#progress loop
#if this step is forgotten, loop will start from the last starting point. nothing is lost. 
#we can recover loop progress simply by executing this block anytime

new_index=i+1
print(new_index)

107


In [8]:
#OR, choose to jump loop points
for j,itm in wl_dict.items():
    print(j, itm)

new_index= ????????????

0 657.0
1 657.1
2 657.2
3 657.3000000000001
4 657.4000000000001
5 657.5000000000001
6 657.6000000000001
7 657.7000000000002
8 657.8000000000002
9 657.9000000000002
10 658.0000000000002
11 658.1000000000003
12 658.2000000000003
13 658.3000000000003
14 658.4000000000003
15 658.5000000000003
16 658.6000000000004
17 658.7000000000004
18 658.8000000000004
19 658.9000000000004
20 659.0000000000005
21 659.1000000000005
22 659.2000000000005
23 659.3000000000005
24 659.4000000000005
25 659.5000000000006
26 659.6000000000006
27 659.7000000000006
28 659.8000000000006
29 659.9000000000007
30 660.0000000000007
31 660.1000000000007
32 660.2000000000007
33 660.3000000000008
34 660.4000000000008
35 660.5000000000008
36 660.6000000000008
37 660.7000000000008
38 660.8000000000009
39 660.9000000000009
40 661.0000000000009
41 661.1000000000009
42 661.200000000001
43 661.300000000001
44 661.400000000001
45 661.500000000001
46 661.600000000001
47 661.7000000000011
48 661.8000000000011
49 661.9000000000011
5

## <span style='color:red'> Option 4: </span> Power feedback loop (in development)

In [ ]:
def converge(x,yy,callback,tol=0.2,max_it=50,rate=0.7):
    it = 0
    y = callback(x)
    while it < max_it:
        err = (yy-y)/yy
        if err < tol:
            break
        x += rate*err*x
        y = callback(x)
        it += 1
    return x,y,err
def set_nkt_power(x):
    set_power(x)
    time.sleep(0.5)
    y = read_power()
    return y
x,y,err = converge(50,500e-9,set_nkt_power)
print(x,y,err)

#### RF power Turn OFF

In [37]:
RF_turnOFF(comport,RF_power)

Turn off RF Power: 0:RegResultSuccess


#### Extreme Turn OFF

In [38]:
Extreme_turnOFF(comport, extreme)

Setting emission OFF - Extreme: 0:RegResultSuccess


In [39]:
max_wl=registerReadU32(comport, RF_power, 0x35,-1)
print(max_wl)
min_wl=registerReadU32(comport, RF_power, 0x34,-1)
print(min_wl)

(0, 670000)
(0, 430000)


## <span style='color:red'> Option 5: </span> dummy host for ANDOR

In [168]:
# Connection to Andor
HOST_andor='127.0.0.1'  #Andor Client C++ program "camera_operate"
PORT_andor=27015
s_andor=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s_andor.bind((HOST_andor,PORT_andor))
s_andor.listen(1)
andor_conn, andor_address = s_andor.accept()
print("Connection from: " + str(andor_address))

Connection from: ('127.0.0.1', 57309)


In [169]:
N=3

for i in range (N):
#  sending go to andor
    flag_andor=andor_conn.send(b"go")
    msg_andor=andor_conn.recv(1024)
    if msg_andor.rstrip(b'\x00').decode("utf-8")=="abc": 
        print(f'andor ok, done-> {i}')
        time.sleep(0.5)
        
print("finished!!!")



andor ok, done-> 0
andor ok, done-> 1
andor ok, done-> 2
finished!!!


## <span style='color:red'> Option 6: </span> dummy host for OSA only

In [72]:
###OSA settings

# ##for vis
# start_WL=645.0  #OSA window start
# stop_WL=677.0  #OSA window end
# RLevel_nW=10

# ##overall
# start_WL=600.0  #OSA window start
# stop_WL=1000.0  #OSA window end
# RLevel_nW=10


##for nir
start_WL=740.0  #OSA window start
stop_WL=812.0  #OSA window end
RLevel_nW=10


Resln=1.0
Sampling_interval= 0.2
Average_times=1
Spn=50

sensitivity="high2"  #Options- norm, mid, high1,2,3. generally use high1
smoothing="OFF"

In [74]:
#Connect to OSA
HOST = '192.168.0.1'    # The remote server (OSA)
PORT = 10001              # The same port as used by the server (OSA)
s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))    
print("OSA connected")
data = sock_send_arr(s,["open \"anonymous\"", " "],receive=True)


OSA connected


In [75]:
start_index=31
N_traces=3

In [76]:
for i in range(start_index,start_index+N_traces ):
    print(f"running --> test{i}") 
    data = sock_send_arr(s,["*RST","CFORM1",f":sens:wav:star {start_WL}nm",f":sens:wav:stop {stop_WL}nm"])
    data=sock_send_arr(s,["CFORM1",f":sens:band:res {Resln}nm",f":sens:sens {sensitivity}"])
    data=sock_send_arr(s,[f":sens:sett:smo {smoothing}",\
                            ":sens:sweep:points:auto off",f":sens:sweep:step {Sampling_interval}nm",\
                          ":sens:corr:rvel:med air"])
    data=sock_send_arr(s,[f":sens:aver:coun {Average_times}",":sens:chop OFF"])
    data=sock_send_arr(s,[":DISPLAY:TRACE:Y1:SPACING LIN",f":DISPLAY:TRACE:Y1:RLEVEL {RLevel_nW}nW"])
#     OSA_data['index']=i
#     OSA_data['setpoint']=wl
    data = sock_send_arr(s,[":init:smode 1","*CLS",":init"])
    data = sock_send(s,":stat:oper:even?",receive=True,cond='1')
    data = sock_send_arr(s,["CFORM1",":mmem:cdr int"])             
    data = sock_send(s,f":mmem:stor:trac tra,csv,\"test_{i}\",int",receive=True)
        
    
flag=s.close()
if flag==None:
    print("OSA Socket Closed Successfully")

running --> test31
running --> test32
running --> test33
OSA Socket Closed Successfully
